이 노트북은 [케라스 창시자에게 배우는 딥러닝 2판](https://tensorflow.blog/kerasdl2/)의 예제 코드를 담고 있습니다.

<table align="left">
    <tr>
        <td>
            <a href="https://colab.research.google.com/github/rickiepark/deep-learning-with-python-2nd/blob/main/chapter11_part04_sequence-to-sequence-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
        </td>
    </tr>
</table>

## 텍스트 분류를 넘어: 시퀀스-투-시퀀스 학습

### 기계 번역 예제

In [1]:
# !wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
# !unzip -q spa-eng.zip

In [14]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

print(text_pairs[:5])

[('Go.', '[start] Ve. [end]'), ('Go.', '[start] Vete. [end]'), ('Go.', '[start] Vaya. [end]'), ('Go.', '[start] Váyase. [end]'), ('Hi.', '[start] Hola. [end]')]


In [15]:
import random
print(random.choice(text_pairs))

("You said it didn't hurt.", '[start] Dijiste que no dolería. [end]')


In [16]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

train_pairs[:5]

[('Give me something to write with.',
  '[start] Deme algo con lo que escribir. [end]'),
 ("What's so funny?", '[start] ¿Qué es tan gracioso? [end]'),
 ('Tom has hardly any close friends.',
  '[start] Tom tiene apenas amigos cercanos. [end]'),
 ('Really?', '[start] ¿En serio? [end]'),
 ("Mary doesn't like it when Tom leaves his dirty clothes on the floor.",
  '[start] A Mary no le gusta cuando Tom deja la ropa sucia en el piso. [end]')]

**영어와 스페인어 텍스트 쌍을 벡터화하기**

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1, # context vector 보다 1개 더 길어야 함. 노션 참고 
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

2024-06-10 06:18:05.268915: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 06:18:11.242808: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 06:18:33.000712: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 06:18:33.041413: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA 

In [18]:
source_vectorization.get_vocabulary()[:10], target_vectorization.get_vocabulary()[:10]


(['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he'],
 ['', '[UNK]', '[start]', '[end]', 'de', 'que', 'a', 'no', 'tom', 'la'])

**번역 작업을 위한 데이터셋 준비하기**

In [19]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

train_ds, val_ds

(<CacheDataset element_spec=({'english': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'spanish': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>,
 <CacheDataset element_spec=({'english': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'spanish': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>)

In [7]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


2024-06-10 02:11:30.395968: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-06-10 02:11:30.397087: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### RNN을 사용한 시퀀스-투-시퀀스 모델

**기반 인코더**

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

**GRU 기반 디코더와 엔드-투-엔드 모델**

In [9]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**RNN 기반 시퀀스-투-시퀀스 모델 훈련하기**

In [10]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15


2024-06-10 02:11:33.749217: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


1302/1302 ━━━━━━━━━━━━━━━━━━━━ 53s 38ms/step - accuracy: 0.1593 - loss: 5.2655 - val_accuracy: 0.1561 - val_loss: 3.8866
Epoch 2/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 48s 37ms/step - accuracy: 0.1595 - loss: 3.8972 - val_accuracy: 0.1887 - val_loss: 3.2483
Epoch 3/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 46s 35ms/step - accuracy: 0.1856 - loss: 3.3303 - val_accuracy: 0.2076 - val_loss: 2.8706
Epoch 4/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 48s 37ms/step - accuracy: 0.2026 - loss: 2.9492 - val_accuracy: 0.2210 - val_loss: 2.6187
Epoch 5/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 46s 36ms/step - accuracy: 0.2162 - loss: 2.6624 - val_accuracy: 0.2313 - val_loss: 2.4429
Epoch 6/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 48s 37ms/step - accuracy: 0.2281 - loss: 2.4324 - val_accuracy: 0.2388 - val_loss: 2.3123
Epoch 7/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 49s 37ms/step - accuracy: 0.2382 - loss: 2.2372 - val_accuracy: 0.2452 - val_loss: 2.2129
Epoch 8/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 50s 38ms/step - accuracy: 0.2472 - loss: 2.07

**RNN 인코더와 디코더로 새로운 문장 번역하기**

In [11]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
I thought Tom wouldn't come.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[start] pensé que tom no vendría [end]
-
If you want a new bike, you'd better start saving up.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[start] si quieres un nuevo bicicleta te [UNK] [end]
-
Only members of the club are entitled to use this room.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/

### 트랜스포머를 사용한 시퀀스-투-시퀀스 모델

#### 트랜스포머 디코더

**`TransformerDecoder` 클래스**

In [12]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def call(self, inputs, encoder_outputs, mask=None):
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            use_causal_mask=True)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### 기계 번역을 위한 트랜스포머

**`PositionalEmbedding` 층**

In [16]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    # def compute_mask(self, inputs, mask=None):
    #     return tf.math.not_equal(inputs, 0)
    def compute_mask(self, inputs, mask=None):
        return tf.keras.layers.Lambda(lambda x: tf.not_equal(x, 0))(inputs)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**엔드-투-엔드 트랜스포머**

In [17]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [18]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

/data/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'transformer_encoder' (of type TransformerEncoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


**시퀀스-투-시퀀스 트랜스포머 훈련하기**

In [19]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30


/data/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/data/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/data/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
I0000 00:00:1717986477.963521   92342 s

   3/1302 ━━━━━━━━━━━━━━━━━━━━ 1:03 49ms/step - accuracy: 0.1435 - loss: 9.3639      

I0000 00:00:1717986485.589091   92342 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 428/1302 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.1326 - loss: 5.5072

W0000 00:00:1717986489.577972   92346 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1717986490.935996    8855 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_82', 152 bytes spill stores, 132 bytes spill loads

I0000 00:00:1717986491.626659    8874 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_60', 16 bytes spill stores, 12 bytes spill loads

I0000 00:00:1717986491.895286    8859 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_60', 128 bytes spill stores, 128 bytes spill loads

I0000 00:00:1717986492.106140    8861 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_84', 152 bytes spill stores, 132 bytes spill loads

I0000 00:00:1717986492.210488    888

1302/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1419 - loss: 4.7814

W0000 00:00:1717986503.953140   92348 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1717986505.192764   92338 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1717986506.296437    9258 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_10', 264 bytes spill stores, 264 bytes spill loads

I0000 00:00:1717986506.306561    9247 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_10', 28 bytes spill stores, 20 bytes spill loads

I0000 00:00:1717986506.404164    9275 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_10', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1717986506.554412    9261 asm_co

1302/1302 ━━━━━━━━━━━━━━━━━━━━ 33s 17ms/step - accuracy: 0.1419 - loss: 4.7808 - val_accuracy: 0.2305 - val_loss: 2.7449
Epoch 2/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.2326 - loss: 2.7729 - val_accuracy: 0.2609 - val_loss: 2.1877
Epoch 3/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.2598 - loss: 2.2787 - val_accuracy: 0.2708 - val_loss: 2.0180
Epoch 4/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.2722 - loss: 2.0583 - val_accuracy: 0.2741 - val_loss: 1.9908
Epoch 5/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.2803 - loss: 1.9316 - val_accuracy: 0.2775 - val_loss: 1.9442
Epoch 6/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.2857 - loss: 1.8514 - val_accuracy: 0.2792 - val_loss: 1.9401
Epoch 7/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.2895 - loss: 1.7987 - val_accuracy: 0.2778 - val_loss: 1.9906
Epoch 8/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.2928 - loss: 1.76

**트랜스포머 모델을 사용해 새로운 문장 번역하기**

In [20]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
School bores me.
[start] la escuela me puedo una escuela [end]
-
How did it go last night?
[start] cómo fue la noche [end]
-
She began doing her homework immediately after dinner.
[start] ella comenzó a hacer inmediatamente después de cenar [end]
-
Do you really think Tom is better than me?
[start] de verdad piensas que tom es mejor que yo [end]
-
I stayed home because I had a bad cold.
[start] me quedé en casa porque estaba frío [end]
-
Tom had a major accident.
[start] tom se [UNK] un accidente [end]
-
These trees are beautiful.
[start] estos árboles son hermosa [end]
-
Suddenly, I blacked out.
[start] de repente me [UNK] [end]
-
That's what I need to find out.
[start] eso es lo que tengo que encontrar la forma [end]
-
There is a tall man in the classroom.
[start] hay un hombre alto en la sala [end]
-
I don't have any pencils.
[start] no tengo ningún lápiz [end]
-
We have been studying English for three years now.
[start] hemos estado estudiando inglés durante tres años [end]
-
Giv

## 요약